# MLOPs Zoomcamp 2023 / 01-homework
By Valerie G., May 2023

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
def get_samples(df_cp, dataset_type, dv=DictVectorizer()):
    df = df_cp.copy()  # to avoid warning about changing original df
    print('dataset_type:', dataset_type)
    num_columns = len(df.columns)  # for homework answer
    print('num_columns:', num_columns)
    print('num_rows:', len(df))
    
    # calc duration and add to df
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)  # convert to min
    
    # for homework answers
    if dataset_type == 'train':
        duration_std = df.duration.std()
        percentage_not_outliers = ((df.duration >= 1) & (df.duration <= 60)).mean() * 100

    # remove outliers
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    # pic feature columns and convert them to dictionaries
    location_colns = ['PULocationID', 'DOLocationID']
    df[location_colns] = df[location_colns].astype(str)
    input_dicts = df[location_colns].to_dict(orient='records')
    
    # convert dict to input features for the model
    if dataset_type == 'train':
        inputs = dv.fit_transform(input_dicts)
    else:
        inputs = dv.transform(input_dicts)
    print('inputs.shape', inputs.shape)
        
    # pic target column and get its values
    target_coln = 'duration'
    targets = df[target_coln].values
    print('targets.shape', targets.shape)
    
    # print homework answers
    if dataset_type == 'train': 
        print('\nQ1:', num_columns)
        print('Q2:', duration_std)
        print('Q3:', percentage_not_outliers)
        matrix_dim = len(inputs.shape)
        print('Q4:', matrix_dim, '\n')
    
    return inputs, targets, dv


def evaluate_model(model, inputs, targets):
    preds = model.predict(inputs)  # predict
    rmse = mean_squared_error(targets, preds, squared=False)  # evaluate
    return rmse

In [ ]:
# read data
df_train = pd.read_parquet('./yellow_tripdata_2022-01.parquet')
df_val = pd.read_parquet('./yellow_tripdata_2022-02.parquet')

# show data
df_train

In [ ]:
# load dataset for model
X_train, y_train, dv = get_samples(df_train, dataset_type='train')
X_val, y_val, _ = get_samples(df_val, dataset_type='val', dv=dv)

Output:
```
dataset_type: train
num_columns: 19
num_rows: 2463931
inputs.shape (2421440, 515)
targets.shape (2421440,)

Q1: 19
Q2: 46.44530513776499
Q3: 98.27547930522405
Q4: 2 

dataset_type: val
num_columns: 19
num_rows: 2979431
inputs.shape (2918187, 515)
targets.shape (2918187,)
```

In [ ]:
# train model
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
# evaluate
rmse_train = evaluate_model(model, X_train, y_train)  # on train set
print('Q5:', rmse_train)
rmse_val = evaluate_model(model, X_val, y_val)  # on validation set
print('Q6:', rmse_val)

Output:
```
Q5: 6.98619083458946
Q6: 7.78640785205543
```